# UniFi Local API Token Setup

Automated setup of UniFi Local API Token using Selenium/Playwright browser automation.

**Purpose:** Create and store a UniFi Local API Token in 1Password Beastmaster vault for accessing the local controller API to get device and client data (thermostats, TVs, computers, etc.).

**💡 Debugging Tip:** For better visualization, run the "Launch Notebook in Browser" cell at the end to open this notebook in a browser with full automation hooks!


In [1]:
# Install required libraries
import sys
!{sys.executable} -m pip install selenium playwright requests python-dotenv matplotlib --quiet
!{sys.executable} -m playwright install chromium
print("✓ Libraries installed (Selenium + Playwright)")


✓ Libraries installed (Selenium + Playwright)


## Architecture Diagram

```mermaid
graph TB
    subgraph "Automation Flow"
        Notebook["📓 This Notebook<br/>Selenium Automation"]
        Selenium["🌐 Selenium/Playwright<br/>Browser<br/>Automation"]
        UniFi["🏠 UniFi OS Controller<br/>https://192.168.1.1"]
    end
    
    subgraph "Token Storage"
        OnePass["🔐 1Password CLI<br/>Beastmaster Vault"]
        EnvFile["📝 ~/.env<br/>(Optional Fallback)"]
    end
    
    subgraph "Usage"
        LocalAPI["🔧 Local Controller API<br/>/proxy/network/api/*"]
        Devices["📱 Devices & Clients<br/>Thermostats, TVs, Computers"]
    end
    
    Notebook -->|"Launch Browser"| Selenium
    Selenium -->|"Login & Navigate"| UniFi
    UniFi -->|"Generate Token"| Selenium
    Selenium -->|"Extract Token"| Notebook
    
    Notebook -->|"Store Token"| OnePass
    Notebook -.->|"Optional"| EnvFile
    
    OnePass -->|"Retrieve Token"| LocalAPI
    LocalAPI -->|"Fetch Data"| Devices
    
    style Notebook fill:#e1f5ff
    style Selenium fill:#fff4e1
    style OnePass fill:#e8f5e9
    style LocalAPI fill:#fce4ec
```


In [2]:
# Load credentials from 1Password
import subprocess
import json
import os
from pathlib import Path
from dotenv import load_dotenv

print("="*70)
print("LOADING CREDENTIALS FROM 1PASSWORD")
print("="*70)

# Load from 1Password Beastmaster vault
def load_unifi_credentials():
    """Load UniFi credentials from 1Password Beastmaster vault."""
    credentials = {}
    
    items = {
        'UNIFI_USERNAME': ('UniFi Username', 'username'),
        'UNIFI_PASSWORD': ('UniFi Password', 'password'),
    }
    
    for env_var, (item_name, field_name) in items.items():
        try:
            result = subprocess.run(
                ['op', 'item', 'get', item_name, '--vault', 'Beastmaster',
                 '--fields', field_name, '--format', 'json', '--reveal'],
                capture_output=True,
                text=True,
                timeout=10
            )
            
            if result.returncode == 0:
                try:
                    field_data = json.loads(result.stdout)
                    if isinstance(field_data, list) and len(field_data) > 0:
                        value = field_data[0].get('value', '')
                    elif isinstance(field_data, dict):
                        value = field_data.get('value', '') or field_data.get(field_name, '')
                    else:
                        value = result.stdout.strip()
                    
                    # Check if value is actually a revealed secret or just a placeholder
                    if value:
                        # 1Password CLI returns placeholder strings when field isn't revealed
                        if value.startswith("[use 'op item get") or "--reveal" in value.lower():
                            print(f"  ⚠️  {item_name}: Field not revealed. Got placeholder string.")
                            print(f"     This means the --reveal flag didn't work. Try manually:")
                            print(f"     op item get '{item_name}' --vault Beastmaster --fields {field_name} --reveal")
                        else:
                            credentials[env_var] = value
                except json.JSONDecodeError:
                    value = result.stdout.strip()
                    # Check if value is actually a revealed secret or just a placeholder
                    if value and not value.startswith("[use 'op item get") and not "--reveal" in value.lower():
                        credentials[env_var] = value
                    elif value.startswith("[use 'op item get"):
                        print(f"  ⚠️  {item_name}: Field not revealed (JSON parse failed but got placeholder)")
        except Exception as e:
            print(f"  ⚠️  Could not load {item_name}: {e}")
    
    return credentials

creds = load_unifi_credentials()

# Fall back to .env if needed
if not creds:
    env_path = Path.home() / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        if os.getenv('UNIFI_USERNAME'):
            creds['UNIFI_USERNAME'] = os.getenv('UNIFI_USERNAME')
        if os.getenv('UNIFI_PASSWORD'):
            creds['UNIFI_PASSWORD'] = os.getenv('UNIFI_PASSWORD')

print(f"\n✓ Loaded {len(creds)} credential(s) from 1Password")
if creds:
    print(f"  • Username: {'✓' if 'UNIFI_USERNAME' in creds else '✗'}")
    print(f"  • Password: {'✓' if 'UNIFI_PASSWORD' in creds else '✗'}")
else:
    print("  ⚠️  No credentials found - add to Beastmaster vault first")


LOADING CREDENTIALS FROM 1PASSWORD
  ⚠️  Could not load UniFi Username: Command '['op', 'item', 'get', 'UniFi Username', '--vault', 'Beastmaster', '--fields', 'username', '--format', 'json', '--reveal']' timed out after 10 seconds
  ⚠️  Could not load UniFi Password: Command '['op', 'item', 'get', 'UniFi Password', '--vault', 'Beastmaster', '--fields', 'password', '--format', 'json', '--reveal']' timed out after 10 seconds

✓ Loaded 2 credential(s) from 1Password
  • Username: ✓
  • Password: ✓


## Clean Up Browser Processes (Optional)

If browser processes are hanging around, run this cell to clean them up.


In [3]:
# Automated token creation using Playwright (Async API for Jupyter)
from playwright.async_api import async_playwright
import asyncio
import time
import base64
from datetime import datetime

print("="*70)
print("AUTOMATED TOKEN CREATION")
print("="*70)

UNIFI_CONTROLLER_URL = "https://192.168.1.1"  # Adjust if needed

# Debugging helpers
DEBUG_MODE = True  # Set to False to reduce output
SCREENSHOT_ON_ERROR = True  # Take screenshots on errors for debugging

def log_step(step_num, message, status="ℹ️"):
    """Log a step with timestamp for debugging."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {status} Step {step_num}: {message}")

async def take_screenshot(page, name):
    """Take a screenshot for debugging."""
    if SCREENSHOT_ON_ERROR:
        try:
            screenshot_path = f"/tmp/unifi_token_{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
            await page.screenshot(path=screenshot_path)
            print(f"  📸 Screenshot saved: {screenshot_path}")
        except Exception as e:
            print(f"  ⚠️  Could not take screenshot: {e}")

async def automate_token_creation():
    """Async function to automate token creation."""
    if not creds.get('UNIFI_USERNAME') or not creds.get('UNIFI_PASSWORD'):
        print("\n⚠️  Missing credentials!")
        print("   Please add UniFi Username and Password to Beastmaster vault.")
        return
    
    print("\n🚀 Launching browser automation...")
    print("   Browser will open - you can watch the automation!")
    print("   (Browser stays open for debugging)")
    print("\n💡 Debugging enabled:")
    print("   • Step-by-step logging")
    print("   • Screenshots on errors")
    print("   • Element inspection")
    print("   • Tell me if something fails and I'll help fix it!\n")
    
    async with async_playwright() as p:
        # Launch browser (visible for debugging)
        # Use persistent context to keep browser alive longer
        log_step("PREP", "Launching Chromium browser")
        browser = await p.chromium.launch(
            headless=False,
            slow_mo=500,
            args=[
                '--start-maximized',
                '--disable-blink-features=AutomationControlled',  # Less detectable as automation
            ]
        )
        
        # Create context with scrolling enabled
        # Note: Removing viewport constraint allows natural scrolling
        context = await browser.new_context(
            ignore_https_errors=True,  # Bypass SSL cert warning (expected for local UniFi)
            # Don't set viewport - let browser use native window size with scroll bars
        )
        page = await context.new_page()
        
        # Force scrollbars to always be visible (even if content fits)
        # This ensures users can always scroll to find menus
        await page.add_init_script("""
            const style = document.createElement('style');
            style.textContent = `
                html {
                    overflow-y: scroll !important;
                    overflow-x: scroll !important;
                }
                body {
                    overflow-y: scroll !important;
                    overflow-x: scroll !important;
                }
                * {
                    scrollbar-width: auto !important;
                    -ms-overflow-style: auto !important;
                }
                ::-webkit-scrollbar {
                    display: block !important;
                    width: 12px !important;
                    height: 12px !important;
                }
            `;
            document.head.appendChild(style);
        """)
        
        # Set longer timeouts
        page.set_default_timeout(30000)  # 30 second default timeout
        page.set_default_navigation_timeout(30000)  # 30 second navigation timeout
        log_step("PREP", "Browser launched", "✅")
        
        try:
            # Navigate to UniFi OS
            log_step(1, f"Navigating to {UNIFI_CONTROLLER_URL}")
            await page.goto(UNIFI_CONTROLLER_URL, wait_until='networkidle', timeout=30000)
            log_step(1, "Navigation complete", "✅")
            
            # Force scrollbars to be visible on the current page
            # (in addition to init script, inject directly after page load)
            await page.evaluate("""
                const style = document.createElement('style');
                style.id = 'force-scrollbars';
                style.textContent = `
                    html { overflow-y: scroll !important; overflow-x: scroll !important; }
                    body { overflow-y: scroll !important; overflow-x: scroll !important; }
                    * { scrollbar-width: auto !important; -ms-overflow-style: auto !important; }
                    ::-webkit-scrollbar { display: block !important; width: 12px !important; height: 12px !important; }
                `;
                if (!document.getElementById('force-scrollbars')) {
                    document.head.appendChild(style);
                }
            """)
            
            # Take initial screenshot
            await take_screenshot(page, "initial_page")
            
            # Wait for login form
            log_step(2, "Waiting for login page to load")
            await asyncio.sleep(2)
            
            # Try to find login elements
            username_selector = "input[type='email'], input[name='username'], input[placeholder*='email'], input[placeholder*='Email']"
            password_selector = "input[type='password']"
            
            try:
                log_step(3, "Looking for login form elements")
                username_input = await page.wait_for_selector(username_selector, timeout=10000)
                password_input = await page.wait_for_selector(password_selector, timeout=10000)
                log_step(3, "Login form elements found", "✅")
                
                log_step(4, "Filling login credentials")
                # Store credentials in local variables to ensure proper dereferencing
                username_value = creds.get('UNIFI_USERNAME', '')
                password_value = creds.get('UNIFI_PASSWORD', '')
                
                if not username_value or not password_value:
                    print("  ❌ ERROR: Credentials not found in creds dictionary!")
                    print(f"     creds keys: {list(creds.keys())}")
                    raise ValueError("Missing credentials in creds dictionary")
                
                print(f"  Debug: Username value = '{username_value[:3]}...' (length: {len(username_value)})")
                print(f"  Debug: Password value length = {len(password_value)}")
                
                await username_input.fill(username_value)
                await password_input.fill(password_value)
                log_step(4, "Credentials filled", "✅")
                await take_screenshot(page, "credentials_filled")
                
                # Find and click submit button
                log_step(5, "Looking for submit button")
                submit_button = await page.wait_for_selector(
                    "button[type='submit'], button:has-text('Sign In'), button:has-text('Login')",
                    timeout=5000
                )
                log_step(5, "Submit button found, clicking...", "✅")
                await submit_button.click()
                
                # Wait for navigation
                log_step(6, "Waiting for login to complete")
                await page.wait_for_load_state('networkidle', timeout=30000)
                log_step(6, "Login successful!", "✅")
                await take_screenshot(page, "after_login")
                
                # Try to automate navigation to API Tokens
                log_step(7, "Attempting to navigate to API Tokens section...")
                
                # Wait a moment for the dashboard to load
                await asyncio.sleep(3)
                
                # Try common navigation patterns for UniFi OS
                navigation_attempted = False
                
                # Try 1: Look for Settings/Settings icon
                try:
                    settings_selectors = [
                        "button:has-text('Settings')",
                        "a:has-text('Settings')",
                        "[aria-label*='Settings']",
                        "[data-testid*='settings']",
                        "svg[aria-label*='Settings']",
                        ".settings-button",
                        "[class*='Settings']",
                        "button[aria-label*='Settings' i]",
                    ]
                    
                    for selector in settings_selectors:
                        try:
                            settings_button = await page.wait_for_selector(selector, timeout=2000)
                            if settings_button:
                                print(f"  ✓ Found settings button via: {selector}")
                                await settings_button.click()
                                navigation_attempted = True
                                await asyncio.sleep(2)
                                # Re-inject scrollbar CSS after navigation
                                await page.evaluate("""
                                    const style = document.createElement('style');
                                    style.id = 'force-scrollbars';
                                    style.textContent = `
                                        html { overflow-y: scroll !important; overflow-x: scroll !important; }
                                        body { overflow-y: scroll !important; overflow-x: scroll !important; }
                                        * { scrollbar-width: auto !important; -ms-overflow-style: auto !important; }
                                        ::-webkit-scrollbar { display: block !important; width: 12px !important; height: 12px !important; }
                                    `;
                                    if (!document.getElementById('force-scrollbars')) {
                                        document.head.appendChild(style);
                                    }
                                """)
                                await take_screenshot(page, "after_settings_click")
                                break
                        except:
                            continue
                except Exception as e:
                    print(f"  ⚠️  Settings navigation attempt: {e}")
                
                # Try 2: Look for User/Account menu
                if not navigation_attempted:
                    try:
                        account_selectors = [
                            "button:has-text('Account')",
                            "a:has-text('Account')",
                            "button:has-text('User')",
                            "[aria-label*='Account']",
                            "[aria-label*='User']",
                            ".account-menu",
                            "[class*='Account']",
                            "[data-testid*='account']",
                            "button[aria-label*='Account' i]",
                            "button[aria-label*='User' i]",
                        ]
                        
                        for selector in account_selectors:
                            try:
                                account_button = await page.wait_for_selector(selector, timeout=2000)
                                if account_button:
                                    print(f"  ✓ Found account/user button via: {selector}")
                                    await account_button.click()
                                    navigation_attempted = True
                                    await asyncio.sleep(2)
                                    # Re-inject scrollbar CSS after navigation
                                    await page.evaluate("""
                                        const style = document.createElement('style');
                                        style.id = 'force-scrollbars';
                                        style.textContent = `
                                            html { overflow-y: scroll !important; overflow-x: scroll !important; }
                                            body { overflow-y: scroll !important; overflow-x: scroll !important; }
                                            * { scrollbar-width: auto !important; -ms-overflow-style: auto !important; }
                                            ::-webkit-scrollbar { display: block !important; width: 12px !important; height: 12px !important; }
                                        `;
                                        if (!document.getElementById('force-scrollbars')) {
                                            document.head.appendChild(style);
                                        }
                                    """)
                                    await take_screenshot(page, "after_account_click")
                                    break
                            except:
                                continue
                    except Exception as e:
                        print(f"  ⚠️  Account navigation attempt: {e}")
                
                # Try 3: Look for Profile/User menu (usually top right)
                if not navigation_attempted:
                    try:
                        profile_selectors = [
                            "button[aria-label*='Profile' i]",
                            "button[aria-label*='Menu' i]",
                            "[class*='Profile']",
                            "[class*='UserMenu']",
                            "[data-testid*='profile']",
                            ".user-menu",
                            ".profile-menu",
                            "button:has(svg[aria-label*='user'])",
                            "button:has(svg[aria-label*='profile'])",
                        ]
                        
                        for selector in profile_selectors:
                            try:
                                profile_button = await page.wait_for_selector(selector, timeout=2000)
                                if profile_button:
                                    print(f"  ✓ Found profile/user menu via: {selector}")
                                    await profile_button.click()
                                    navigation_attempted = True
                                    await asyncio.sleep(2)
                                    # Re-inject scrollbar CSS after navigation
                                    await page.evaluate("""
                                        const style = document.createElement('style');
                                        style.id = 'force-scrollbars';
                                        style.textContent = `
                                            html { overflow-y: scroll !important; overflow-x: scroll !important; }
                                            body { overflow-y: scroll !important; overflow-x: scroll !important; }
                                            * { scrollbar-width: auto !important; -ms-overflow-style: auto !important; }
                                            ::-webkit-scrollbar { display: block !important; width: 12px !important; height: 12px !important; }
                                        `;
                                        if (!document.getElementById('force-scrollbars')) {
                                            document.head.appendChild(style);
                                        }
                                    """)
                                    await take_screenshot(page, "after_profile_click")
                                    break
                            except:
                                continue
                    except Exception as e:
                        print(f"  ⚠️  Profile navigation attempt: {e}")
                
                # Try 4: Look for API Tokens directly in the page
                try:
                    api_token_selectors = [
                        "button:has-text('API Token')",
                        "a:has-text('API Token')",
                        "button:has-text('API Tokens')",
                        "[aria-label*='API Token']",
                        "[data-testid*='api-token']",
                        ".api-token",
                        "[class*='ApiToken']",
                        "*:has-text('API Token')",
                    ]
                    
                    for selector in api_token_selectors:
                        try:
                            api_token_button = await page.wait_for_selector(selector, timeout=2000)
                            if api_token_button:
                                print(f"  ✓ Found API Token button directly via: {selector}")
                                await api_token_button.click()
                                navigation_attempted = True
                                await asyncio.sleep(2)
                                await take_screenshot(page, "after_api_token_click")
                                break
                        except:
                            continue
                except Exception as e:
                    print(f"  ⚠️  Direct API Token search: {e}")
                
                # Take a screenshot of current state
                await take_screenshot(page, "current_state_before_manual")
                
                # Get current URL and page content for debugging
                current_url = page.url
                print(f"\n  📍 Current URL: {current_url}")
                
                # Try to get page title and some visible text
                try:
                    page_title = await page.title()
                    print(f"  📄 Page title: {page_title}")
                except:
                    pass
                
                # Detailed analysis of what's on the Settings page
                print(f"\n  🔍 Analyzing Settings page structure...")
                try:
                    # Get all clickable elements with their text
                    all_clickable = await page.query_selector_all("button, a, [role='button'], [role='link'], [tabindex='0']")
                    visible_items = []
                    for elem in all_clickable[:50]:  # Check first 50
                        try:
                            text = (await elem.inner_text()).strip()
                            # Get aria-label as fallback
                            aria_label = await elem.get_attribute("aria-label")
                            if aria_label:
                                text = text or aria_label
                            if text and len(text) < 100:
                                visible_items.append(text)
                        except:
                            pass
                    
                    # Remove duplicates while preserving order
                    seen = set()
                    unique_items = []
                    for item in visible_items:
                        if item.lower() not in seen:
                            seen.add(item.lower())
                            unique_items.append(item)
                    
                    if unique_items:
                        print(f"\n  📋 All clickable items on Settings page ({len(unique_items)} found):")
                        for i, text in enumerate(unique_items[:25], 1):  # Show first 25
                            # Highlight anything that might be related to API/Token/Account
                            marker = ""
                            if any(keyword in text.lower() for keyword in ['api', 'token', 'account', 'user', 'profile', 'settings', 'advanced', 'system']):
                                marker = " ⭐"
                            print(f"      {i:2d}. {text}{marker}")
                        
                        # Look specifically for API/Token related items
                        api_token_candidates = [item for item in unique_items if 'api' in item.lower() or 'token' in item.lower()]
                        if api_token_candidates:
                            print(f"\n  🎯 API/Token related items found:")
                            for item in api_token_candidates:
                                print(f"      • {item}")
                        
                        # Look for Account/User/Profile items
                        account_candidates = [item for item in unique_items if any(kw in item.lower() for kw in ['account', 'user', 'profile'])]
                        if account_candidates:
                            print(f"\n  👤 Account/User/Profile items found:")
                            for item in account_candidates:
                                print(f"      • {item}")
                    
                except Exception as e:
                    print(f"  ⚠️  Could not analyze page structure: {e}")
                
                # Also try to get all text content to see what's visible
                try:
                    body_text = await page.inner_text("body")
                    # Look for keywords in the page text
                    keywords = ['API Token', 'API Tokens', 'Token', 'Account', 'User Settings', 'Profile', 'Advanced', 'System']
                    found_keywords = []
                    for keyword in keywords:
                        if keyword.lower() in body_text.lower():
                            found_keywords.append(keyword)
                    if found_keywords:
                        print(f"\n  🔑 Keywords found in page text: {', '.join(found_keywords)}")
                except:
                    pass
                
                if navigation_attempted:
                    print("\n  ✅ Navigation attempted - check screenshot to see where we are")
                else:
                    print("\n  ⚠️  Could not find navigation path automatically")
                
                print("\n" + "="*70)
                print("👆 HUMAN ACTION REQUIRED 👆")
                print("="*70)
                print("\n🥪 YOU ARE NOW IN THE MIDDLE OF THE SANDWICH 🥪")
                print("   (Human-in-the-middle step - your action needed!)")
                print("\n📋 Manual Navigation Required:")
                print("   The automation tried to find navigation paths but may need your help.")
                print("   Check the screenshots saved to /tmp/unifi_token_*.png to see current state")
                print("\n   1. Look for Settings, Account, or Profile menu")
                print("   2. Navigate to: Settings → Account → API Tokens")
                print("      OR: User Menu → Settings → API Tokens")
                print("      OR: Profile → API Tokens")
                print("   3. Click 'Create New Token' or 'Add Token'")
                print("   4. Name it: 'beast-unifi-integration-token'")
                print("   5. Copy the token when shown (you'll only see it once!)")
                print("\n⚠️  IMPORTANT: Browser window has scroll bars - you can scroll to find menus")
                print("📸 Screenshots saved to /tmp/unifi_token_*.png for reference")
                print("\n✅ Once you've copied the token, press Enter in this notebook...")
                print("="*70)
                input()
                
                # Try to extract token from page
                log_step(7, "Extracting token from page")
                await asyncio.sleep(2)
                
                # Look for token in various formats
                token = None
                token_selectors = [
                    "input[readonly][value], input[type='text'][readonly]",
                    "code, pre, .token, [class*='token']",
                    "textarea[readonly]",
                ]
                
                for selector in token_selectors:
                    try:
                        elements = await page.query_selector_all(selector)
                        for elem in elements:
                            text = (await elem.inner_text()).strip()
                            if text and len(text) > 20:
                                token = text
                                print(f"  ✓ Token found via {selector}")
                                break
                        if token:
                            break
                    except:
                        continue
                
                if not token:
                    print("  ⚠️  Could not auto-extract token")
                    print("  Please copy the token from the browser and paste below:")
                    token = input("\nPaste token here: ").strip()
                
                if token and len(token) > 10:
                    print(f"\n✅ Token extracted! ({len(token)} characters)")
                    print(f"   Preview: {token[:10]}...{token[-4:]}")
                    
                    # Store in 1Password
                    print("\n7. Storing token in 1Password Beastmaster vault...")
                    
                    token_item = {
                        "title": "UniFi Local API Token",
                        "category": "API_CREDENTIAL",
                        "fields": [
                            {
                                "id": "password",
                                "value": token,
                                "purpose": "PASSWORD",
                                "label": "Token"
                            },
                            {
                                "id": "notesPlain",
                                "value": "UniFi Local Controller API Token\n\nCreated for beast-unifi-integration project.\nUsed to access local controller at https://192.168.1.1\nfor fetching device and client data (thermostats, TVs, computers, etc.).",
                                "label": "Notes"
                            }
                        ]
                    }
                    
                    result = subprocess.run(
                        ['op', 'item', 'create', '--vault', 'Beastmaster', '--format', 'json'],
                        input=json.dumps(token_item),
                        text=True,
                        capture_output=True,
                        timeout=10
                    )
                    
                    if result.returncode == 0:
                        print("  ✅ Token stored in Beastmaster vault!")
                    else:
                        print(f"  ⚠️  Error storing: {result.stderr[:200]}")
                        print("  💡 Token extracted - you can store it manually if needed")
                    
            except Exception as e:
                log_step("ERROR", f"Automation error occurred: {e}", "❌")
                await take_screenshot(page, f"error_{datetime.now().strftime('%H%M%S')}")
                
                print("\n" + "="*70)
                print("🐛 DEBUGGING INFO")
                print("="*70)
                print(f"\n❌ Error: {e}")
                print(f"❌ Error type: {type(e).__name__}")
                print(f"\n📸 Screenshot saved (check /tmp/unifi_token_*.png)")
                
                # Check if browser/page is still alive
                try:
                    if browser.is_connected():
                        print(f"\n🌐 Browser still connected")
                        print(f"🌐 Current URL: {page.url}")
                        try:
                            title = await page.title()
                            print(f"📄 Page title: {title}")
                        except:
                            print(f"📄 Could not get page title")
                        
                        # Try to get page content snippet for debugging
                        try:
                            page_text = (await page.inner_text('body'))[:500]
                            print(f"\n📝 Page content preview (first 500 chars):")
                            print(f"   {page_text[:200]}...")
                        except:
                            print("📝 Could not get page content")
                    else:
                        print(f"\n⚠️  Browser disconnected (this might be why it closed)")
                except Exception as e2:
                    print(f"\n⚠️  Could not check browser status: {e2}")
                
                print("\n💡 What to tell me for debugging:")
                print("   1. What page are you on? (screenshot saved)")
                print("   2. What error did you see?")
                print("   3. Did the browser close? When?")
                print("   4. Can you manually navigate to API Tokens?")
                print("\n   I'll update the notebook with fixes!")
                
                print("\n💡 Browser should still be open - you can complete the process manually:")
                print("   1. Log in to UniFi OS (if not already)")
                print("   2. Navigate to Settings → API Tokens")
                print("   3. Create new token")
                print("   4. Copy the token")
                print("\n   Then run the 'Manual Token Entry' cell to store it.")
                print("\n⏸️  Browser will stay open... (close it when done with manual steps)")
        
        finally:
            # Keep browser open for debugging - don't close immediately
            print("\n" + "="*70)
            print("💡 BROWSER STAYING OPEN FOR DEBUGGING")
            print("="*70)
            print("\n✅ Browser window should still be open")
            print("   • You can interact with it manually")
            print("   • Navigate to API Tokens if needed")
            print("   • Create token manually if automation didn't complete")
            print("\n🔧 To keep browser open longer:")
            print("   • The browser will stay open until you close it manually")
            print("   • Or until you interrupt this cell")
            print("\n💡 If automation didn't complete:")
            print("   1. Navigate manually in the browser window")
            print("   2. Create the token manually")
            print("   3. Run the 'Manual Token Entry' cell to store it")
            
            # Keep browser open - wait for user to close it
            # This way the browser doesn't close immediately
            try:
                print("\n⏸️  Keeping browser open... (Close browser window when done)")
                print("   Press Ctrl+C in this cell if you want to proceed")
                # Wait while browser is still connected - check more frequently
                wait_count = 0
                while browser.is_connected():
                    await asyncio.sleep(0.5)  # Check every 0.5 seconds
                    wait_count += 1
                    # Print status every 10 seconds
                    if wait_count % 20 == 0:
                        print(f"   Browser still open... ({wait_count//2} seconds)")
                print("\n✓ Browser closed by user")
            except KeyboardInterrupt:
                print("\n⚠️  Interrupted - closing browser...")
                if browser.is_connected():
                    await browser.close()
                print("   Browser closed")
            except Exception as e:
                print(f"\n⚠️  Browser check error: {e}")
                # Make sure browser is actually closed
                try:
                    if browser.is_connected():
                        await browser.close()
                        print("   Browser force-closed due to error")
                except:
                    pass

# Run the async function
print("\n🚀 Starting automation...\n")
await automate_token_creation()


AUTOMATED TOKEN CREATION

🚀 Starting automation...


🚀 Launching browser automation...
   Browser will open - you can watch the automation!
   (Browser stays open for debugging)

💡 Debugging enabled:
   • Step-by-step logging
   • Screenshots on errors
   • Element inspection
   • Tell me if something fails and I'll help fix it!

[10:13:30] ℹ️ Step PREP: Launching Chromium browser
[10:13:32] ✅ Step PREP: Browser launched
[10:13:32] ℹ️ Step 1: Navigating to https://192.168.1.1
[10:13:37] ✅ Step 1: Navigation complete
  📸 Screenshot saved: /tmp/unifi_token_initial_page_20251103_101337.png
[10:13:37] ℹ️ Step 2: Waiting for login page to load
[10:13:39] ℹ️ Step 3: Looking for login form elements
[10:13:39] ✅ Step 3: Login form elements found
[10:13:39] ℹ️ Step 4: Filling login credentials
  Debug: Username value = 'lou...' (length: 19)
  Debug: Password value length = 15
[10:13:40] ✅ Step 4: Credentials filled
  📸 Screenshot saved: /tmp/unifi_token_credentials_filled_20251103_101340.png
[1

CancelledError: 

## Manual Token Entry (Fallback)

If automation didn't work or you can't find API Tokens in the local interface, use this cell.

**Note**: API Tokens may only be available via:
- **UniFi Cloud Portal**: https://unifi.ui.com (log in, go to Settings → API Tokens)
- **May require admin/owner role** (not just a regular user)
- **May require specific UniFi OS version**

If you created the token via the cloud portal or another method, paste it here.


In [ ]:
# Manual token entry (if automation didn't work)
print("="*70)
print("MANUAL TOKEN ENTRY")
print("="*70)

print("\nIf automation didn't work, you can manually create the token:")
print("1. Open https://192.168.1.1 in your browser")
print("2. Log in")
print("3. Go to Settings → API Tokens")
print("4. Create new token")
print("5. Copy the token")
print("\nThen paste it below:")

manual_token = input("\nPaste token here (or press Enter to skip): ").strip()

if manual_token and len(manual_token) > 10:
    print(f"\n✓ Token received ({len(manual_token)} characters)")
    
    # Store in 1Password
    print("\nStoring in 1Password Beastmaster vault...")
    
    token_item = {
        "title": "UniFi Local API Token",
        "category": "API_CREDENTIAL",
        "fields": [
            {
                "id": "password",
                "value": manual_token,
                "purpose": "PASSWORD",
                "label": "Token"
            },
            {
                "id": "notesPlain",
                "value": "UniFi Local Controller API Token\n\nCreated for beast-unifi-integration project.",
                "label": "Notes"
            }
        ]
    }
    
    result = subprocess.run(
        ['op', 'item', 'create', '--vault', 'Beastmaster', '--format', 'json'],
        input=json.dumps(token_item),
        text=True,
        capture_output=True,
        timeout=10
    )
    
    if result.returncode == 0:
        print("✅ Token stored in Beastmaster vault!")
    else:
        print(f"⚠️  Error: {result.stderr[:200]}")
else:
    print("\n⚠️  No token entered")


## Verify Token

Test that the token works by connecting to the local controller and fetching sample data.


In [ ]:
# Verify token works
print("="*70)
print("VERIFYING TOKEN")
print("="*70)

# Get token from 1Password
try:
    result = subprocess.run(
        ['op', 'item', 'get', 'UniFi Local API Token', '--vault', 'Beastmaster', '--fields', 'password'],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0 and result.stdout.strip():
        token = result.stdout.strip()
        print(f"\n✓ Token retrieved from 1Password ({len(token)} chars)")
        
        # Test connection
        print("\n📡 Testing connection to local controller...")
        
        try:
            from beast_unifi.api.local_controller import LocalControllerClient
            
            client = LocalControllerClient(
                base_url="https://192.168.1.1:443",
                api_token=token,
                site="default"
            )
            
            # Test with sites endpoint
            sites = client.get_sites()
            print(f"  ✅ Connected! Found {len(sites)} site(s)")
            
            # Try to get devices
            print("\n📱 Fetching devices...")
            devices = client.get_devices()
            print(f"  ✓ Found {len(devices)} UniFi device(s)")
            
            # Try to get clients
            print("\n👥 Fetching clients...")
            clients = client.get_clients()
            print(f"  ✓ Found {len(clients)} client(s)")
            
            if clients:
                print("\n📋 Sample clients (first 10):")
                for client_data in clients[:10]:
                    hostname = client_data.get('hostname', 'Unknown')
                    ip = client_data.get('ip', 'N/A')
                    mac = client_data.get('mac', 'N/A')
                    print(f"     • {hostname} - IP: {ip}, MAC: {mac}")
                
                # Look for interesting devices
                interesting = []
                for client_data in clients:
                    hostname = client_data.get('hostname', '').lower()
                    if any(keyword in hostname for keyword in ['thermostat', 'tv', 'tv-', 'computer', 'pc', 'laptop']):
                        interesting.append(client_data)
                
                if interesting:
                    print("\n⭐ Interesting devices found:")
                    for client_data in interesting:
                        hostname = client_data.get('hostname', 'Unknown')
                        ip = client_data.get('ip', 'N/A')
                        print(f"     • {hostname} - IP: {ip}")
            
            print("\n✅ Token is working!")
            print("   You can now use it to fetch device/client data.")
            
        except Exception as e:
            print(f"  ✗ Connection failed: {e}")
            print("  💡 Check that:")
            print("     • Controller is accessible")
            print("     • Token is valid")
            print("     • Token has correct permissions")
    else:
        print("\n⚠️  Token not found in 1Password")
except Exception as e:
    print(f"\n⚠️  Error: {e}")


## Launch Notebook in Browser for Better Visualization & Debugging

This cell uses Playwright to launch the notebook in a browser with **full automation hooks**, making it much better for:
- 📊 **Viewing visualizations** clearly (Mermaid diagrams)
- 🐛 **Debugging** browser automation interactively
- 🎯 **Stepping through cells** and watching automation in action
- 🔧 **Using browser developer tools** for inspection

**This is the meta-document debugging tool!**


In [ ]:
# Launch notebook in browser using Playwright for better visualization & debugging
# This is the "meta-document" debugging tool - using the notebook itself for debugging!

from playwright.async_api import async_playwright
import subprocess
import asyncio
import sys
from pathlib import Path

print("="*60)
print("LAUNCHING NOTEBOOK IN BROWSER FOR DEBUGGING")
print("="*60)

notebook_file = Path("unifi_local_token_setup.ipynb")
notebook_name = notebook_file.name

async def launch_notebook_browser():
    if not notebook_file.exists():
        print(f"✗ Notebook not found: {notebook_file}")
        return
    
    print(f"✓ Found notebook: {notebook_name}")
    
    # Check if Jupyter server is running
    jupyter_url = None
    
    try:
        import requests
        for port in [8888, 8889, 8890, 8887]:
            try:
                response = requests.get(f'http://localhost:{port}', timeout=2)
                if response.status_code == 200:
                    jupyter_url = f"http://localhost:{port}"
                    print(f"✓ Jupyter server already running on {jupyter_url}")
                    break
            except:
                pass
    except ImportError:
        print("ℹ requests not available - will start new server")
    
    if not jupyter_url:
        print("\n🚀 Starting Jupyter server...")
        print("   (This may take a moment)")
        
        # Start Jupyter server
        jupyter_cmd = [sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888']
        jupyter_process = subprocess.Popen(
            jupyter_cmd,
            cwd=Path.cwd(),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait for server to start
        print("   Waiting for server to start...")
        for i in range(15):
            await asyncio.sleep(1)
            try:
                import requests
                response = requests.get('http://localhost:8888', timeout=1)
                if response.status_code == 200:
                    jupyter_url = "http://localhost:8888"
                    print(f"✓ Server started on {jupyter_url}")
                    break
            except:
                if i % 3 == 0:
                    print(f"   Still waiting... ({i+1}/15)")
        
        if not jupyter_url:
            print("⚠️  Server may need more time - trying anyway...")
            jupyter_url = "http://localhost:8888"
            await asyncio.sleep(2)
    
    if jupyter_url:
        # Jupyter expects relative path from server root
        # The notebook is at notebooks/examples/unifi_local_token_setup.ipynb
        notebook_path = "notebooks/examples/unifi_local_token_setup.ipynb"
        notebook_url = f"{jupyter_url}/notebooks/{notebook_path}"
        print(f"\n🌐 Launching notebook with Playwright browser automation...")
        print(f"   URL: {notebook_url}\n")
        
        try:
            async with async_playwright() as p:
                # Launch browser with full automation hooks
                print("🚀 Launching Chromium browser with automation hooks...")
                browser = await p.chromium.launch(
                    headless=False,
                    args=['--start-maximized']
                )
                
                # Create context with large viewport for better visualization
                context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
                page = await context.new_page()
                
                # Navigate to notebook
                print("📓 Loading notebook...")
                await page.goto(notebook_url, wait_until='networkidle', timeout=30000)
                
                print("✓ Notebook opened in browser with full automation hooks!")
                print("\n✅ Browser window is now open - you can:")
                print("   📊 View all visualizations clearly (Mermaid diagrams)")
                print("   🐛 Debug browser automation interactively")
                print("   🎯 Step through automation cells")
                print("   🔧 Use Playwright hooks for inspection")
                print(f"\n📝 Server: {jupyter_url}")
                print(f"🌐 Notebook: {notebook_url}")
                print("\n💡 Browser will stay open - interact with it freely")
                print("   Close the browser window when you're done")
                print("\n⏸️  This is your debugging meta-document!")
                print("   Use it to step through and debug the automation interactively\n")
                
                # Keep browser open for interaction
                try:
                    while browser.is_connected():
                        await asyncio.sleep(1)
                except KeyboardInterrupt:
                    print("\n\n⚠️  Interrupted - closing browser...")
                
                if browser.is_connected():
                    await browser.close()
                print("✓ Browser closed")
                
        except Exception as e:
            print(f"⚠️  Playwright error: {e}")
            print(f"\n💡 Opening with regular browser instead...")
            import webbrowser
            webbrowser.open(notebook_url)
            print(f"✓ Notebook opened in default browser")
    else:
        print("⚠️  Could not start Jupyter server")
        print(f"   Try manually: {sys.executable} -m jupyter notebook")

# Run the async function
await launch_notebook_browser()


## Debugging & Collaboration Tools

**🤝 How We Collaborate:**

While you run this notebook in the browser:

1. **You run cells** and watch automation happen
2. **If something fails**, tell me:
   - What step failed (the log shows step numbers)
   - What error you saw
   - What the page looks like
   - Screenshot location (saved to `/tmp/unifi_token_*.png`)
3. **I'll update the notebook** with fixes/debugging improvements
4. **You re-run** the updated cells
5. **We iterate** until it works!

**🔧 Available Debugging Tools:**

- ✅ **Step-by-step logging** with timestamps
- ✅ **Automatic screenshots** on errors
- ✅ **Element inspection** (see what elements are found)
- ✅ **Page content preview** for debugging
- ✅ **Current URL/title** tracking
- ✅ **Interactive debugging** (browser stays open)


In [ ]:
# Debugging helper: Inspect current page state
# Run this cell if automation fails to get debugging info

from playwright.sync_api import sync_playwright

print("="*70)
print("PAGE INSPECTION TOOL")
print("="*70)
print("\n💡 This helps me understand what's on the page if automation fails")
print("   Run this while the browser is still open from the automation cell")

# Check if there's a running browser context we can inspect
# (This would require the browser to still be open from previous cell)

print("\n📋 What I need from you if automation fails:")
print("   1. Current URL of the page")
print("   2. Page title")
print("   3. Screenshot (check /tmp/unifi_token_*.png)")
print("   4. What elements are visible? (login form, buttons, etc.)")
print("   5. Any error messages on the page?")

print("\n💡 Or you can:")
print("   • Take a screenshot manually")
print("   • Inspect elements with browser dev tools")
print("   • Tell me what you see and I'll update the selectors!")

# Helper to test selectors
def test_selectors():
    """Test if selectors are working."""
    print("\n🔍 Testing common selectors...")
    # This would need the page object from the automation
    # But we can guide manual testing
    selectors_to_test = [
        "input[type='email']",
        "input[name='username']",
        "input[type='password']",
        "button[type='submit']",
    ]
    
    print("   Test these in browser console:")
    for selector in selectors_to_test:
        print(f"   document.querySelector('{selector}')")
    
    print("\n   Or use Playwright's inspector:")
    print("   PWDEBUG=1 python -m playwright codegen https://192.168.1.1")

test_selectors()


## Overview

This notebook automates the process of:
1. **Logging into UniFi OS** at your local controller
2. **Navigating to API Tokens settings**
3. **Creating a new API token**
4. **Storing the token** in 1Password Beastmaster vault
5. **Verifying** the token works

**Why Local API Token?**

**⚠️ SSL Certificate Warning:**

When accessing your local UniFi controller (192.168.1.1), your browser will show a security warning because the controller uses a self-signed certificate. **This is completely normal and expected** for local UniFi controllers.

- The automation will automatically bypass this warning
- You may see a "Privacy error" or "Not secure" warning
- This is safe to ignore for local network access
- The automation handles it automatically

- Site Manager API only gives gateway devices (2 hosts)
- Local API gives ALL network devices and clients
- Required for accessing: thermostats, TVs, computers, phones, etc.